## Setting up a performance analysis software environment

This notebook guides you in the installation of the minimal set of software packages required to run the Simple notebook. It assumes that you have C and C++ compilers installed, e.g., gcc.

1. Create a project directory, e.g, `$HOME/performance`, and continue the following steps inside that project directory, `$PROJECT_DIR`.

2. Install [TAU Commander](http://taucommander.paratools.com/). This also installs Miniconda3 and several python packages we will need later.
```
cd $PROJECT_DIR
git clone --branch unstable https://github.com/ParaToolsInc/taucmdr.git taucmdr-unstable
cd taucmdr-unstable
make install 
```


 You should add the TAU Commander binary directory to your path:
```
export PATH=$PROJECT_DIR/taucmdr/bin:$PATH
```

3. Configure your Conda environment for the newly installed Miniconda3 (under your taucmdr directory): 

```
MYSHELL=`basename $SHELL`
$PROJECT_DIR/taucmdr/installed/conda/bin/conda init $MYSHELL
exec $MYSHELL
$PROJECT_DIR/taucmdr/installed/conda/bin/conda install matplotlib
```

4. Install Hatchet:
```
cd $PROJECT_DIR
git clone https://github.com/LLNL/hatchet.git
cd hatchet
./install.sh
```

5. Test your installation: